# Лабораторная работа №1 по предмету "Теория кодирования".
Выполнили студенты группы 6404-010302D,
Журавская Елизавета и Кандалов Иван.



In [18]:
import numpy as np
# 1.1. Реализовать функцию REF(), приводящую матрицу к ступенчатому виду.
def REF(matrix):
    # Приведение матрицы к ступенчатому виду
    rows, cols = matrix.shape
    lead = 0 #lead используется для отслеживания текущего столбца, в котором мы ищем ведущий элемент (первый ненулевой элемент в строке)

    for r in range(rows):
        if lead >= cols: #Если lead превышает количество столбцов, мы выходим из цикла, так как больше нет столбцов для поиска
            break
        i = r
        while matrix[i, lead] == 0:
            i += 1
            if i == rows:
                i = r
                lead =+ 1
                if cols == lead:
                    return matrix[:r]  # Возвращаем только ненулевые строки
        matrix[[i, r]] = matrix[[r, i]]  # Меняем местами строки
        for i in range(r + 1, rows): #есть ведущий элемент в строке r, мы обнуляем все элементы ниже него в том же столбце.
            if matrix[i, lead] == 1:  # Если элемент ненулевой
                matrix[i] ^= matrix[r]  # XOR каждой строки с той строкой, которую мы выбрали (в которой есть ненулевой элемент)
        lead += 1 #Переход к следующему столбцу

    return matrix[:r]  # Возвращаем только ненулевые строки

# Пример использования
matrix = np.array([[0, 0, 1, 0, 1],
                  [0, 0, 0, 1, 0],
                  [0, 1, 0, 1, 1],
                  [0, 1, 0, 1, 0]], dtype=int)

result = REF(matrix)
print(result)



[[0 1 0 1 1]
 [0 0 1 0 1]
 [0 0 0 1 0]]


In [19]:
# 1.2. Реализовать функцию RREF(), приводящую матрицу к приведённому ступенчатому виду.
def RREF(matrix):
    rows, cols = matrix.shape
    lead = 0 #lead используется для отслеживания текущего столбца, в котором мы ищем ведущий элемент (первый ненулевой элемент в строке)

    for r in range(rows):
        if lead >= cols:
            return matrix
        # Находим первую ненулевую строку в текущем столбце
        i = r
        while matrix[i, lead] == 0:
            i += 1
            if i == rows:
                i = r
                lead += 1
                if lead == cols:
                    return matrix
        # Меняем местами строки
        matrix[[i, r]] = matrix[[r, i]]
        # Нормализуем ведущий элемент (делаем его равным 1)
        if matrix[r, lead] != 0:
            matrix[r] = matrix[r] / matrix[r, lead]
        # Обнуляем все элементы ниже и выше ведущего элемента в этом столбце
        for i in range(rows):
            if i != r and matrix[i, lead] == 1: #проверяем, есть ли ненулевой элемент в текущем столбце lead, кроме выбранного r
                matrix[i] = matrix[i] ^ matrix[r]
        lead += 1
    return matrix

# Пример использования
matrix = np.array([[0, 0, 1, 0, 1],
                   [0, 0, 0, 1, 0],
                   [0, 1, 0, 1, 1],
                   [0, 1, 0, 1, 0]], dtype=int)

result = RREF(matrix)
print(result)


[[0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]


In [20]:
# 1.3. Создать класс линейных кодов LinearCode.
# 1.3.1 На основе входной матрицы сформировать порождающую матрицу в ступенчатом виде.
S = np.array([[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
             [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
             [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
             [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
             [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
             [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]])
S_REF = RREF(S)
print(S_REF)

[[1 0 1 0 0 1 0 1 0 1 0]
 [0 0 0 1 0 1 0 0 0 1 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0]]


In [21]:
# 1.3.2 Задать n равное числу столбцов и k равное числу строк полученной матрицы (без учёта полностью нулевых строк).
non_zero_rows = np.any(S_REF, axis=1) #создаем массив булевых значений, где `True` соответствует строкам, которые содержат хотя бы один ненулевой элемент
S_REF_new = S_REF[non_zero_rows] #индексируем матрицу `S_REF`, выбирая только строки с `True` в массиве `non_zero_rows`
n = S_REF_new.shape[1] # Число столбцов
k = S_REF_new.shape[0] # Число строк без учёта нулевых
print('n =', n)
print('k =', k)

n = 11
k = 5


In [22]:
# 1.3.3 Сформировать проверочную матрицу на основе порождающей.
# Шаг 1. Сформировать матрицу 𝐆∗ в приведённом ступенчатом виде на основе порождающей.
G = np.array([[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
              [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
              [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])
G_REF = RREF(G)
print(G_REF)

[[1 0 1 0 0 1 0 1 0 1 0]
 [0 0 0 1 0 1 0 0 0 1 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1]]


In [23]:
# Шаг 2. Зафиксировать ведущие столбцы lead матрицы 𝐆∗.
def find_leading_columns(matrix):
    rows, cols = matrix.shape
    leading_cols = [] # Список для хранения номеров ведущих столбцов
    current_row = 0 # Текущая строка для поиска ведущего элемента
    current_col = 0 # Текущий столбец для поиска ведущего элемента

    while current_row < rows and current_col < cols:
        # Ищем ведущий элемент в текущей строке
        while current_col < cols and matrix[current_row, current_col] == 0:
            current_col += 1
        # Если ведущий элемент найден, добавляем номер столбца в список
        if current_col < cols:
            leading_cols.append(current_col)
        # Переходим к следующей строке и столбцу
        current_row += 1
        current_col += 1
    return leading_cols

leading_cols = find_leading_columns(G_REF)
print(f"Номера ведущих столбцов: {leading_cols}")

Номера ведущих столбцов: [0, 3, 4, 6, 8]


In [24]:
# Шаг 3. Сформировать сокращённую матрицу 𝐗, удалив ведущие столбцы матрицы 𝐆∗.
keep_indices = list(set(range(G_REF.shape[1])) - set(leading_cols)) #G_REF.shape[1]-количество столбцов
#range(G_REF.shape[1]) создает объект, который представляет последовательность чисел индексов всех столбцов в матрице
#set()- превращает последовательность индексов в множество
X = G_REF[:, keep_indices]
print(X)

[[0 1 1 1 1 0]
 [0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [0 0 0 0 1 0]
 [0 0 0 0 1 1]]


In [25]:
# Создаем единичную матрицу I
I = np.eye(len(X[0]), dtype = int)
print(I)

[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


In [26]:
#Шаг 4. Сформировать матрицу 𝐇, поместив в строки, соответствующие позициям ведущих столбцов строки из 𝐗, а в остальные – строки единичной матрицы.

H = []
j = 0
r = 0
for i in range(len(X)+len(I)):
    if i in leading_cols:
        H.append(X[r])
        r+=1
    else:
        H.append(I[j])
        j+=1
print(*H)

[0 1 1 1 1 0] [1 0 0 0 0 0] [0 1 0 0 0 0] [0 0 1 0 1 1] [0 0 0 1 0 1] [0 0 1 0 0 0] [0 0 0 0 1 0] [0 0 0 1 0 0] [0 0 0 0 1 1] [0 0 0 0 1 0] [0 0 0 0 0 1]


In [27]:
# Шаг 4. Сформировать матрицу 𝐇, поместив в строки, соответствующие позициям ведущих столбцов строки из 𝐗, а в остальные – строки единичной матрицы.
H = np.zeros((len(X)+len(I), len(X[0])), dtype = int)
j = 0
r = 0
for i in range(len(X)+len(I)):
    if i in leading_cols:
        H[i,:]=X[r,:]
        r+=1
    else:
        H[i,:] = I[j,:]
        j+=1
print(H)

[[0 1 1 1 1 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


In [28]:
# 1.4. Сформировать все кодовые слова длины n двумя способами.
# 1.4.1 Сложить все слова из порождающего множества, оставить неповторяющиеся.
G = np.array([[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
              [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
              [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])

H = np.array([[0, 1, 1, 1, 1, 0],
             [1, 0, 0, 0, 0, 0],
             [0, 1, 0, 0, 0, 0],
             [0, 0, 1, 0, 1, 1],
             [0, 0, 0, 1, 0, 1],
             [0, 0, 1, 0, 0, 0],
             [0, 0, 0, 0, 1, 0],
             [0, 0, 0, 1, 0, 0],
             [0, 0, 0, 0, 1, 1],
             [0, 0, 0, 0, 1, 0],
             [0, 0, 0, 0, 0, 1]])


In [29]:
from itertools import combinations
M = np.array([[0, 1, 0, 0],  # строка 1
              [0, 0, 1, 1],  # строка 2
              [1, 1, 0, 0]]) # строка 4

# Получаем все комбинации строк
n = M.shape[0]
xor_results = []
spisok=[]
# Выполняем XOR для всех комбинаций из 2 и более строк
for r in range(2, n + 1):  # от 2 до n строк (включительно)
    for indices in combinations(range(n), r):  # получаем все комбинации
        result = np.zeros(M.shape[1], dtype=int)  # инициализируем результат
        for index in indices:  # для каждой выбранной строки
            result = np.bitwise_xor(result, M[index])
        xor_results.append(result)


print(*xor_results)

[0 1 1 1] [1 0 0 0] [1 1 1 1] [1 0 1 1]


In [30]:
# 1.4.2 Взять все двоичные слова длины k, умножить каждое на G.

u = np.array([1, 0, 1, 1, 0])
print('u = ', u)

v = np.dot(u,G)
v %= 2
print('v = u@G ', v)

check = np.dot(v, H)
check %= 2
print('v@H =', check)

u =  [1 0 1 1 0]
v = u@G  [1 0 1 1 1 0 1 0 0 1 0]
v@H = [0 0 0 0 0 0]


In [31]:
# 1.5 Вычислить кодовое расстояние получившегося кода.

from itertools import combinations

def find_code_distance(matrix):
    min_weight = float('inf') #верхнее значение для сравнения

    # Перебираем все комбинации строк матрицы
    for r in range(1, len(matrix) + 1):
        for combo in combinations(matrix, r):
            # Инициализируем суммированный вектор
            combined = np.zeros(len(matrix[0]))
            # Суммируем строки по модулю 2
            for row in combo:
                for i in range(len(row)):
                    combined[i] = (combined[i] + row[i]) % 2

            # Вычисляем вес полученного вектора
            weight = sum(combined)
            if 0 < weight < min_weight:
                min_weight = weight

    return int(min_weight)

n=len(G[0])
k=len(G)
print('n=', n, '\nk=', k)

# Вычисляем кодовое расстояние
d = find_code_distance(G)

# Вычисляем кратность обнаруживаемой ошибки
t = (d - 1)
#t = (d - 1) // 2
print("d =", d)
print("t =", t)


n= 11 
k= 5
d = 2
t = 1


In [32]:
# 1.4.1 Внести в кодовое слово ошибку кратности не более t, умножить полученное слово на H, убедиться в обнаружении ошибки.
v = np.array([1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0])
e1 = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
Ve1 = (v + e1) % 2
print('v+e1', Ve1)
check = np.dot(Ve1, H)
check %= 2
print('(v+e1)@H =', check, "- error")

v+e1 [1 0 0 1 1 0 1 0 0 1 0]
(v+e1)@H = [0 1 0 0 0 0] - error


In [33]:
# 1.4.2 Найти для некоторого кодового слова ошибку кратности t+1 такую, что при умножении на H ошибка не может быть обнаружена.
def find_e2(v, H):
    # Перебираем все пары позиций в кодовом слове
    for i in range(len(v) - 1):
        for j in range(i + 1, len(v)):
            # Создаем вектор ошибки e2
            #перебираем все возможные пары индексов
            e2 = np.zeros(len(v), dtype=int)
            e2[i] = 1
            e2[j] = 1
            e2 = np.array(e2)
            VE2 = (v + e2) % 2 #Создание вектора с ошибкой
            check = np.dot(VE2, H)
            check %= 2 #Проверка обнаружения ошибки
            a = 0
            for k in range(0, len(check)): #Если a состоит только из нулей, то ошибка не обнаружена
                if check[k] == 0:
                    a += 1
            if a == len(check):
                return e2
    return None

e2 = find_e2(v, H)
print('e2', e2)

Ve2 = (v + e2) % 2
print('v+e2', Ve2)

check = np.dot(Ve2, H)
check %= 2
print('(v+e2)@H =', check, "- no error")

e2 [0 0 0 0 0 0 1 0 0 1 0]
v+e2 [1 0 1 1 1 0 0 0 0 0 0]
(v+e2)@H = [0 0 0 0 0 0] - no error
